# Creating Custom VS Code AI Agent Tools

This comprehensive guide shows how to create custom tools that AI agents can automatically discover and invoke, similar to the built-in tools like `create_file`, `semantic_search`, etc.

## Prerequisites
- Node.js 18+ installed
- VS Code with extension development setup
- TypeScript knowledge
- Understanding of VS Code Extension API

## 1. Setting Up Development Environment

First, let's set up the development environment for creating custom AI agent tools.

In [ ]:
# Install required tools
npm install -g yo generator-code vsce

# Create new extension
yo code
# Choose: New Extension (TypeScript)
# Name: my-ai-tools
# Description: Custom AI Agent Tools

cd my-ai-tools
npm install

## 2. Understanding VS Code Extension Architecture

VS Code AI agent tools work through:
1. **Tool Registration**: Extensions register tools via contribution points
2. **Discovery**: AI agents discover tools through VS Code's API
3. **Invocation**: AI agents call tools with parameters
4. **Response**: Tools return structured results

## 3. Creating Basic Extension Manifest (package.json)

In [ ]:
{
  "name": "my-ai-tools",
  "displayName": "Custom AI Tools",
  "description": "Custom tools for AI agents",
  "version": "1.0.0",
  "engines": {
    "vscode": "^1.85.0"
  },
  "categories": ["Other"],
  "activationEvents": [
    "onStartupFinished"
  ],
  "main": "./out/extension.js",
  "contributes": {
    "aiTools": [
      {
        "name": "custom_file_analyzer",
        "displayName": "Custom File Analyzer",
        "description": "Analyzes files with custom logic",
        "parameters": {
          "type": "object",
          "properties": {
            "filePath": {
              "type": "string",
              "description": "Path to file to analyze"
            },
            "analysisType": {
              "type": "string",
              "enum": ["complexity", "dependencies", "metrics"],
              "description": "Type of analysis to perform"
            }
          },
          "required": ["filePath", "analysisType"]
        }
      }
    ]
  },
  "scripts": {
    "compile": "tsc -p ./",
    "watch": "tsc -watch -p ./"
  },
  "devDependencies": {
    "@types/vscode": "^1.85.0",
    "typescript": "^5.0.0"
  }
}

## 4. Implementing Tool Interface

Create a base interface that all AI tools must implement:

In [ ]:
// src/interfaces/IAITool.ts
export interface IAIToolParameter {
  name: string;
  type: 'string' | 'number' | 'boolean' | 'object' | 'array';
  description: string;
  required: boolean;
  enum?: string[];
  properties?: { [key: string]: IAIToolParameter };
}

export interface IAIToolResult {
  success: boolean;
  data?: any;
  error?: string;
  metadata?: { [key: string]: any };
}

export interface IAITool {
  readonly name: string;
  readonly displayName: string;
  readonly description: string;
  readonly parameters: IAIToolParameter[];

  execute(parameters: { [key: string]: any }): Promise<IAIToolResult>;
  validate(parameters: { [key: string]: any }): boolean;
}

## 5. Tool Registration and Discovery

Create a tool registry that AI agents can discover:

In [ ]:
// src/ToolRegistry.ts
import * as vscode from 'vscode';
import { IAITool, IAIToolResult } from './interfaces/IAITool';

export class ToolRegistry {
  private static instance: ToolRegistry;
  private tools: Map<string, IAITool> = new Map();

  public static getInstance(): ToolRegistry {
    if (!ToolRegistry.instance) {
      ToolRegistry.instance = new ToolRegistry();
    }
    return ToolRegistry.instance;
  }

  public registerTool(tool: IAITool): void {
    this.tools.set(tool.name, tool);
    console.log(`Registered AI tool: ${tool.name}`);
  }

  public getTool(name: string): IAITool | undefined {
    return this.tools.get(name);
  }

  public getAllTools(): IAITool[] {
    return Array.from(this.tools.values());
  }

  public async executeTool(name: string, parameters: any): Promise<IAIToolResult> {
    const tool = this.getTool(name);
    if (!tool) {
      return {
        success: false,
        error: `Tool '${name}' not found`
      };
    }

    try {
      if (!tool.validate(parameters)) {
        return {
          success: false,
          error: 'Invalid parameters'
        };
      }

      return await tool.execute(parameters);
    } catch (error) {
      return {
        success: false,
        error: error instanceof Error ? error.message : 'Unknown error'
      };
    }
  }
}

## 6. Handling AI Agent Communication

Create communication layer for AI agents:

In [ ]:
// src/AIAgentCommunication.ts
import * as vscode from 'vscode';
import { ToolRegistry } from './ToolRegistry';

export class AIAgentCommunication {
  private registry: ToolRegistry;

  constructor() {
    this.registry = ToolRegistry.getInstance();
  }

  public registerCommands(context: vscode.ExtensionContext): void {
    // Register command to list available tools
    const listToolsCmd = vscode.commands.registerCommand(
      'myAiTools.listTools',
      () => {
        const tools = this.registry.getAllTools();
        return tools.map(tool => ({
          name: tool.name,
          displayName: tool.displayName,
          description: tool.description,
          parameters: tool.parameters
        }));
      }
    );

    // Register command to execute tools
    const executeToolCmd = vscode.commands.registerCommand(
      'myAiTools.executeTool',
      async (toolName: string, parameters: any) => {
        return await this.registry.executeTool(toolName, parameters);
      }
    );

    context.subscriptions.push(listToolsCmd, executeToolCmd);
  }

  // API for AI agents to discover tools
  public async onDidDiscoverTools(): Promise<void> {
    const tools = this.registry.getAllTools();

    // Notify VS Code that tools are available
    await vscode.commands.executeCommand(
      'setContext',
      'myAiTools.available',
      tools.length > 0
    );
  }
}

## 7. Creating File Operation Tools

Build custom file operation tools similar to built-in ones:

In [ ]:
// src/tools/CustomFileAnalyzer.ts
import * as vscode from 'vscode';
import * as fs from 'fs';
import * as path from 'path';
import { IAITool, IAIToolParameter, IAIToolResult } from '../interfaces/IAITool';

export class CustomFileAnalyzer implements IAITool {
  public readonly name = 'custom_file_analyzer';
  public readonly displayName = 'Custom File Analyzer';
  public readonly description = 'Analyzes files with custom metrics and insights';

  public readonly parameters: IAIToolParameter[] = [
    {
      name: 'filePath',
      type: 'string',
      description: 'Absolute path to the file to analyze',
      required: true
    },
    {
      name: 'analysisType',
      type: 'string',
      description: 'Type of analysis to perform',
      required: true,
      enum: ['complexity', 'dependencies', 'metrics', 'security']
    }
  ];

  public validate(parameters: { [key: string]: any }): boolean {
    const { filePath, analysisType } = parameters;

    if (!filePath || typeof filePath !== 'string') {
      return false;
    }

    if (!analysisType || !['complexity', 'dependencies', 'metrics', 'security'].includes(analysisType)) {
      return false;
    }

    return true;
  }

  public async execute(parameters: { [key: string]: any }): Promise<IAIToolResult> {
    const { filePath, analysisType } = parameters;

    try {
      if (!fs.existsSync(filePath)) {
        return {
          success: false,
          error: `File not found: ${filePath}`
        };
      }

      const content = fs.readFileSync(filePath, 'utf8');
      const ext = path.extname(filePath);

      let analysisResult;
      switch (analysisType) {
        case 'complexity':
          analysisResult = this.analyzeComplexity(content, ext);
          break;
        case 'dependencies':
          analysisResult = this.analyzeDependencies(content, ext);
          break;
        case 'metrics':
          analysisResult = this.analyzeMetrics(content, ext);
          break;
        case 'security':
          analysisResult = this.analyzeSecurity(content, ext);
          break;
        default:
          throw new Error(`Unsupported analysis type: ${analysisType}`);
      }

      return {
        success: true,
        data: {
          filePath,
          analysisType,
          fileExtension: ext,
          result: analysisResult
        },
        metadata: {
          fileSize: content.length,
          lineCount: content.split('\n').length,
          timestamp: new Date().toISOString()
        }
      };
    } catch (error) {
      return {
        success: false,
        error: error instanceof Error ? error.message : 'Analysis failed'
      };
    }
  }

  private analyzeComplexity(content: string, ext: string): any {
    const lines = content.split('\n');
    const cyclomaticComplexity = this.calculateCyclomaticComplexity(content, ext);

    return {
      cyclomaticComplexity,
      lineCount: lines.length,
      functionCount: this.countFunctions(content, ext),
      nestingDepth: this.calculateMaxNestingDepth(content, ext)
    };
  }

  private analyzeDependencies(content: string, ext: string): any {
    const imports = this.extractImports(content, ext);
    const exports = this.extractExports(content, ext);

    return {
      imports,
      exports,
      externalDependencies: imports.filter(imp => !imp.startsWith('./') && !imp.startsWith('../')),
      internalDependencies: imports.filter(imp => imp.startsWith('./') || imp.startsWith('../'))
    };
  }

  private analyzeMetrics(content: string, ext: string): any {
    return {
      characterCount: content.length,
      lineCount: content.split('\n').length,
      wordCount: content.split(/\s+/).filter(word => word.length > 0).length,
      commentLines: this.countCommentLines(content, ext),
      blankLines: content.split('\n').filter(line => line.trim() === '').length
    };
  }

  private analyzeSecurity(content: string, ext: string): any {
    const securityIssues = [];

    // Check for common security patterns
    if (content.includes('eval(')) {
      securityIssues.push({ type: 'eval-usage', severity: 'high', message: 'Use of eval() detected' });
    }

    if (content.includes('innerHTML')) {
      securityIssues.push({ type: 'innerHTML-usage', severity: 'medium', message: 'Use of innerHTML detected' });
    }

    return {
      securityIssues,
      riskLevel: securityIssues.length > 0 ? 'medium' : 'low'
    };
  }

  // Helper methods
  private calculateCyclomaticComplexity(content: string, ext: string): number {
    // Simplified cyclomatic complexity calculation
    const complexityKeywords = ['if', 'else', 'while', 'for', 'case', 'catch', '&&', '||'];
    let complexity = 1; // Base complexity

    for (const keyword of complexityKeywords) {
      const regex = new RegExp(`\\b${keyword}\\b`, 'g');
      const matches = content.match(regex);
      if (matches) {
        complexity += matches.length;
      }
    }

    return complexity;
  }

  private countFunctions(content: string, ext: string): number {
    const functionRegex = /function\s+\w+|\w+\s*[=:]\s*function|\w+\s*[=:]\s*\([^)]*\)\s*=>/g;
    const matches = content.match(functionRegex);
    return matches ? matches.length : 0;
  }

  private calculateMaxNestingDepth(content: string, ext: string): number {
    let maxDepth = 0;
    let currentDepth = 0;

    for (const char of content) {
      if (char === '{') {
        currentDepth++;
        maxDepth = Math.max(maxDepth, currentDepth);
      } else if (char === '}') {
        currentDepth--;
      }
    }

    return maxDepth;
  }

  private extractImports(content: string, ext: string): string[] {
    const imports = [];
    const importRegex = /import.*from\s+['"]([^'"]+)['"]/g;
    let match;

    while ((match = importRegex.exec(content)) !== null) {
      imports.push(match[1]);
    }

    return imports;
  }

  private extractExports(content: string, ext: string): string[] {
    const exports = [];
    const exportRegex = /export\s+(?:default\s+)?(?:class|function|const|let|var)\s+(\w+)/g;
    let match;

    while ((match = exportRegex.exec(content)) !== null) {
      exports.push(match[1]);
    }

    return exports;
  }

  private countCommentLines(content: string, ext: string): number {
    const lines = content.split('\n');
    let commentCount = 0;

    for (const line of lines) {
      const trimmed = line.trim();
      if (trimmed.startsWith('//') || trimmed.startsWith('/*') || trimmed.startsWith('*')) {
        commentCount++;
      }
    }

    return commentCount;
  }
}

## 8. Building Code Analysis Tools

Create semantic search and code analysis tools:

In [ ]:
// src/tools/CustomSemanticSearch.ts
import * as vscode from 'vscode';
import * as fs from 'fs';
import * as path from 'path';
import { IAITool, IAIToolParameter, IAIToolResult } from '../interfaces/IAITool';

export class CustomSemanticSearch implements IAITool {
  public readonly name = 'custom_semantic_search';
  public readonly displayName = 'Custom Semantic Search';
  public readonly description = 'Advanced semantic search with context understanding';

  public readonly parameters: IAIToolParameter[] = [
    {
      name: 'query',
      type: 'string',
      description: 'Natural language query to search for',
      required: true
    },
    {
      name: 'includePatterns',
      type: 'array',
      description: 'File patterns to include in search',
      required: false
    },
    {
      name: 'excludePatterns',
      type: 'array',
      description: 'File patterns to exclude from search',
      required: false
    },
    {
      name: 'maxResults',
      type: 'number',
      description: 'Maximum number of results to return',
      required: false
    }
  ];

  public validate(parameters: { [key: string]: any }): boolean {
    const { query } = parameters;
    return query && typeof query === 'string' && query.trim().length > 0;
  }

  public async execute(parameters: { [key: string]: any }): Promise<IAIToolResult> {
    const {
      query,
      includePatterns = ['**/*.ts', '**/*.js', '**/*.md'],
      excludePatterns = ['**/node_modules/**', '**/dist/**'],
      maxResults = 50
    } = parameters;

    try {
      const workspaceFolders = vscode.workspace.workspaceFolders;
      if (!workspaceFolders || workspaceFolders.length === 0) {
        return {
          success: false,
          error: 'No workspace folder found'
        };
      }

      const searchResults = await this.performSemanticSearch(
        query,
        workspaceFolders[0].uri.fsPath,
        includePatterns,
        excludePatterns,
        maxResults
      );

      return {
        success: true,
        data: {
          query,
          results: searchResults,
          totalMatches: searchResults.length
        },
        metadata: {
          searchTime: new Date().toISOString(),
          includePatterns,
          excludePatterns
        }
      };
    } catch (error) {
      return {
        success: false,
        error: error instanceof Error ? error.message : 'Search failed'
      };
    }
  }

  private async performSemanticSearch(
    query: string,
    rootPath: string,
    includePatterns: string[],
    excludePatterns: string[],
    maxResults: number
  ): Promise<any[]> {
    const results = [];
    const queryKeywords = this.extractKeywords(query.toLowerCase());

    // Find files matching patterns
    const files = await this.findFiles(rootPath, includePatterns, excludePatterns);

    for (const filePath of files) {
      if (results.length >= maxResults) break;

      try {
        const content = fs.readFileSync(filePath, 'utf8');
        const relevanceScore = this.calculateRelevance(content, queryKeywords);

        if (relevanceScore > 0.1) { // Threshold for relevance
          const matches = this.findContextMatches(content, queryKeywords);

          results.push({
            filePath: path.relative(rootPath, filePath),
            relevanceScore,
            matches,
            preview: this.generatePreview(content, queryKeywords)
          });
        }
      } catch (error) {
        // Skip files that can't be read
      }
    }

    // Sort by relevance score
    return results.sort((a, b) => b.relevanceScore - a.relevanceScore);
  }

  private extractKeywords(query: string): string[] {
    return query
      .split(/\s+/)
      .filter(word => word.length > 2)
      .map(word => word.toLowerCase());
  }

  private calculateRelevance(content: string, keywords: string[]): number {
    const lowerContent = content.toLowerCase();
    let score = 0;

    for (const keyword of keywords) {
      const regex = new RegExp(keyword, 'gi');
      const matches = lowerContent.match(regex);
      if (matches) {
        score += matches.length * (keyword.length / 10); // Weight by keyword length
      }
    }

    return Math.min(score / content.length * 1000, 1); // Normalize to 0-1
  }

  private findContextMatches(content: string, keywords: string[]): any[] {
    const lines = content.split('\n');
    const matches = [];

    for (let i = 0; i < lines.length; i++) {
      const line = lines[i].toLowerCase();
      for (const keyword of keywords) {
        if (line.includes(keyword)) {
          matches.push({
            lineNumber: i + 1,
            content: lines[i].trim(),
            keyword,
            context: this.getContext(lines, i, 2)
          });
        }
      }
    }

    return matches;
  }

  private getContext(lines: string[], lineIndex: number, contextSize: number): string[] {
    const start = Math.max(0, lineIndex - contextSize);
    const end = Math.min(lines.length, lineIndex + contextSize + 1);
    return lines.slice(start, end);
  }

  private generatePreview(content: string, keywords: string[]): string {
    const lines = content.split('\n');
    for (let i = 0; i < lines.length; i++) {
      const line = lines[i].toLowerCase();
      for (const keyword of keywords) {
        if (line.includes(keyword)) {
          const context = this.getContext(lines, i, 1);
          return context.join('\n').substring(0, 200) + '...';
        }
      }
    }
    return content.substring(0, 200) + '...';
  }

  private async findFiles(
    rootPath: string,
    includePatterns: string[],
    excludePatterns: string[]
  ): Promise<string[]> {
    const files: string[] = [];

    const walkDir = (dir: string): void => {
      const items = fs.readdirSync(dir);

      for (const item of items) {
        const fullPath = path.join(dir, item);
        const relativePath = path.relative(rootPath, fullPath);

        // Check exclude patterns
        if (excludePatterns.some(pattern => this.matchGlob(relativePath, pattern))) {
          continue;
        }

        const stat = fs.statSync(fullPath);
        if (stat.isDirectory()) {
          walkDir(fullPath);
        } else if (stat.isFile()) {
          // Check include patterns
          if (includePatterns.some(pattern => this.matchGlob(relativePath, pattern))) {
            files.push(fullPath);
          }
        }
      }
    };

    walkDir(rootPath);
    return files;
  }

  private matchGlob(filePath: string, pattern: string): boolean {
    // Simple glob matching (for production, use a proper glob library)
    const regexPattern = pattern
      .replace(/\*\*/g, '.*')
      .replace(/\*/g, '[^/]*')
      .replace(/\?/g, '.');

    const regex = new RegExp(`^${regexPattern}$`);
    return regex.test(filePath.replace(/\\/g, '/'));
  }
}

## 9. Implementing Terminal Integration

Create tools that interact with VS Code's terminal:

In [ ]:
// src/tools/CustomTerminalTool.ts
import * as vscode from 'vscode';
import { IAITool, IAIToolParameter, IAIToolResult } from '../interfaces/IAITool';

export class CustomTerminalTool implements IAITool {
  public readonly name = 'custom_terminal_executor';
  public readonly displayName = 'Custom Terminal Executor';
  public readonly description = 'Execute commands in terminal with enhanced output capture';

  private terminals: Map<string, vscode.Terminal> = new Map();
  private outputBuffers: Map<string, string[]> = new Map();

  public readonly parameters: IAIToolParameter[] = [
    {
      name: 'command',
      type: 'string',
      description: 'Command to execute in terminal',
      required: true
    },
    {
      name: 'workingDirectory',
      type: 'string',
      description: 'Working directory for command execution',
      required: false
    },
    {
      name: 'terminalName',
      type: 'string',
      description: 'Name of terminal instance to use',
      required: false
    },
    {
      name: 'captureOutput',
      type: 'boolean',
      description: 'Whether to capture command output',
      required: false
    }
  ];

  public validate(parameters: { [key: string]: any }): boolean {
    const { command } = parameters;
    return command && typeof command === 'string' && command.trim().length > 0;
  }

  public async execute(parameters: { [key: string]: any }): Promise<IAIToolResult> {
    const {
      command,
      workingDirectory,
      terminalName = 'AI-Agent-Terminal',
      captureOutput = true
    } = parameters;

    try {
      const terminal = this.getOrCreateTerminal(terminalName, workingDirectory);

      if (captureOutput) {
        this.setupOutputCapture(terminalName);
      }

      // Change directory if specified
      if (workingDirectory) {
        terminal.sendText(`cd "${workingDirectory}"`);
        await this.delay(500); // Wait for directory change
      }

      // Execute command
      terminal.sendText(command);
      terminal.show();

      // Wait for command completion if capturing output
      let output = '';
      if (captureOutput) {
        output = await this.waitForCommandCompletion(terminalName, 10000); // 10 second timeout
      }

      return {
        success: true,
        data: {
          command,
          terminalName,
          workingDirectory,
          output: captureOutput ? output : 'Output capture disabled'
        },
        metadata: {
          executionTime: new Date().toISOString(),
          terminalId: terminal.creationOptions?.name || terminalName
        }
      };
    } catch (error) {
      return {
        success: false,
        error: error instanceof Error ? error.message : 'Command execution failed'
      };
    }
  }

  private getOrCreateTerminal(name: string, cwd?: string): vscode.Terminal {
    let terminal = this.terminals.get(name);

    if (!terminal || terminal.exitStatus) {
      terminal = vscode.window.createTerminal({
        name,
        cwd: cwd || vscode.workspace.workspaceFolders?.[0]?.uri.fsPath
      });
      this.terminals.set(name, terminal);
    }

    return terminal;
  }

  private setupOutputCapture(terminalName: string): void {
    if (!this.outputBuffers.has(terminalName)) {
      this.outputBuffers.set(terminalName, []);
    }
  }

  private async waitForCommandCompletion(terminalName: string, timeout: number): Promise<string> {
    return new Promise((resolve) => {
      const buffer = this.outputBuffers.get(terminalName) || [];
      const startTime = Date.now();

      const checkCompletion = () => {
        if (Date.now() - startTime > timeout) {
          resolve(buffer.join('\n') + '\n[Command timeout]');
          return;
        }

        // Check if command is complete (simplified - in real implementation,
        // you'd need more sophisticated completion detection)
        setTimeout(checkCompletion, 100);
      };

      setTimeout(() => {
        resolve(buffer.join('\n'));
      }, 2000); // Wait 2 seconds for output
    });
  }

  private delay(ms: number): Promise<void> {
    return new Promise(resolve => setTimeout(resolve, ms));
  }

  // Cleanup method to call on extension deactivation
  public dispose(): void {
    for (const terminal of this.terminals.values()) {
      terminal.dispose();
    }
    this.terminals.clear();
    this.outputBuffers.clear();
  }
}

## 10. Main Extension Entry Point

Wire everything together in the main extension file:

In [ ]:
// src/extension.ts
import * as vscode from 'vscode';
import { ToolRegistry } from './ToolRegistry';
import { AIAgentCommunication } from './AIAgentCommunication';
import { CustomFileAnalyzer } from './tools/CustomFileAnalyzer';
import { CustomSemanticSearch } from './tools/CustomSemanticSearch';
import { CustomTerminalTool } from './tools/CustomTerminalTool';

export function activate(context: vscode.ExtensionContext) {
    console.log('Activating Custom AI Tools extension...');

    // Initialize registry and communication
    const registry = ToolRegistry.getInstance();
    const communication = new AIAgentCommunication();

    // Register all custom tools
    const tools = [
        new CustomFileAnalyzer(),
        new CustomSemanticSearch(),
        new CustomTerminalTool()
    ];

    for (const tool of tools) {
        registry.registerTool(tool);
    }

    // Register commands for AI agent communication
    communication.registerCommands(context);

    // Notify that tools are available
    communication.onDidDiscoverTools();

    // Register status bar item
    const statusBarItem = vscode.window.createStatusBarItem(
        vscode.StatusBarAlignment.Right,
        100
    );
    statusBarItem.text = `$(tools) AI Tools: ${tools.length}`;
    statusBarItem.tooltip = 'Custom AI Tools Available';
    statusBarItem.command = 'myAiTools.listTools';
    statusBarItem.show();
    context.subscriptions.push(statusBarItem);

    // Register additional commands
    const showToolsCmd = vscode.commands.registerCommand(
        'myAiTools.showAvailableTools',
        () => {
            const availableTools = registry.getAllTools();
            const toolList = availableTools.map(tool =>
                `**${tool.displayName}** (${tool.name})\n${tool.description}\n`
            ).join('\n');

            vscode.window.showInformationMessage(
                `Available AI Tools:\n\n${toolList}`,
                { modal: true }
            );
        }
    );

    context.subscriptions.push(showToolsCmd);

    console.log(`Custom AI Tools extension activated with ${tools.length} tools`);
}

export function deactivate() {
    console.log('Deactivating Custom AI Tools extension...');

    // Cleanup terminals
    const registry = ToolRegistry.getInstance();
    const terminalTool = registry.getTool('custom_terminal_executor') as CustomTerminalTool;
    if (terminalTool) {
        terminalTool.dispose();
    }
}

## 11. TypeScript Configuration

Set up TypeScript configuration for the extension:

In [ ]:
// tsconfig.json
{
  "compilerOptions": {
    "module": "commonjs",
    "target": "ES2020",
    "outDir": "out",
    "lib": ["ES2020"],
    "sourceMap": true,
    "rootDir": "src",
    "strict": true,
    "esModuleInterop": true,
    "skipLibCheck": true,
    "forceConsistentCasingInFileNames": true,
    "resolveJsonModule": true
  },
  "exclude": ["node_modules", ".vscode-test"]
}

## 12. Testing Your Custom Tools

Create tests to verify tool functionality:

In [ ]:
// src/test/tools.test.ts
import * as assert from 'assert';
import * as vscode from 'vscode';
import { CustomFileAnalyzer } from '../tools/CustomFileAnalyzer';
import { ToolRegistry } from '../ToolRegistry';

suite('Custom AI Tools Tests', () => {
    let registry: ToolRegistry;
    let fileAnalyzer: CustomFileAnalyzer;

    setup(() => {
        registry = ToolRegistry.getInstance();
        fileAnalyzer = new CustomFileAnalyzer();
        registry.registerTool(fileAnalyzer);
    });

    test('Tool Registration', () => {
        const tool = registry.getTool('custom_file_analyzer');
        assert.strictEqual(tool?.name, 'custom_file_analyzer');
        assert.strictEqual(tool?.displayName, 'Custom File Analyzer');
    });

    test('Parameter Validation', () => {
        // Valid parameters
        const validParams = {
            filePath: '/path/to/file.ts',
            analysisType: 'complexity'
        };
        assert.strictEqual(fileAnalyzer.validate(validParams), true);

        // Invalid parameters
        const invalidParams = {
            filePath: '',
            analysisType: 'invalid'
        };
        assert.strictEqual(fileAnalyzer.validate(invalidParams), false);
    });

    test('Tool Execution', async () => {
        // Create a temporary test file
        const testContent = `
function testFunction() {
    if (true) {
        console.log('test');
    }
}
`;

        const tempFile = await vscode.workspace.fs.writeFile(
            vscode.Uri.file('/tmp/test.ts'),
            Buffer.from(testContent)
        );

        const result = await fileAnalyzer.execute({
            filePath: '/tmp/test.ts',
            analysisType: 'complexity'
        });

        assert.strictEqual(result.success, true);
        assert.ok(result.data);
        assert.ok(result.data.result);

        // Cleanup
        await vscode.workspace.fs.delete(vscode.Uri.file('/tmp/test.ts'));
    });

    test('Registry Tool Execution', async () => {
        // Test execution through registry
        const result = await registry.executeTool('unknown_tool', {});
        assert.strictEqual(result.success, false);
        assert.ok(result.error?.includes('not found'));
    });
});

## 13. Building and Packaging

Build and package your extension:

In [ ]:
# Build the extension
npm run compile

# Test the extension
npm test

# Package for distribution
vsce package

# Install locally for testing
code --install-extension my-ai-tools-1.0.0.vsix

# Publish to marketplace (optional)
vsce publish

## 14. Usage and Integration

Once installed, AI agents can discover and use your tools automatically:

### AI Agent Integration

Your custom tools will be available to AI agents through:

1. **Tool Discovery**: AI agents call `myAiTools.listTools` to discover available tools
2. **Parameter Validation**: Tools validate input parameters automatically
3. **Execution**: AI agents call `myAiTools.executeTool` with tool name and parameters
4. **Response Handling**: Tools return structured results with success/error status

### Example AI Agent Usage

```javascript
// AI agent discovers tools
const availableTools = await vscode.commands.executeCommand('myAiTools.listTools');

// AI agent executes custom file analyzer
const result = await vscode.commands.executeCommand(
    'myAiTools.executeTool',
    'custom_file_analyzer',
    {
        filePath: '/workspace/src/app.ts',
        analysisType: 'complexity'
    }
);

if (result.success) {
    console.log('Analysis result:', result.data);
} else {
    console.error('Analysis failed:', result.error);
}
```

### Best Practices

1. **Comprehensive Parameter Validation**: Always validate inputs thoroughly
2. **Structured Error Handling**: Return consistent error formats
3. **Performance Optimization**: Implement timeouts and resource limits
4. **Documentation**: Provide clear descriptions and examples
5. **Testing**: Create comprehensive test suites
6. **Security**: Validate file paths and prevent harmful operations